In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import struct                                                                                                    
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import uproot
import os

def bread(filename, position, size, endianess):
  with open(filename, 'rb') as f:
    bdata = []
    cnt = 0
    f.seek(position,0)
    
    while cnt < size:
      bytes = f.read(2)
      if bytes == b'':
        break
      else:
        if endianess == "little":
            bdata.append(struct.unpack('<H', bytes)[0]) # 2bytes
        else:
            bdata.append(struct.unpack('>H', bytes)[0]) # 2bytes
            
      cnt += 1  
  hdata = [hex(x) for x in bdata]
  return bdata, hdata

def seek_header(filename):
    offset = 0
    with open(filename, 'rb') as f:
        bdata = []
        while True:
          bytes = f.read(2)
          if bytes == b'':
            break
          elif bytes == b'\x90\xeb' or bytes == b'\xeb\x90':
            break
          else:
            offset += 2
    return offset

def seek_endianess(filename):
    print("Reading data file {0}".format(filename))
    endianess = 0
    offset = 0
    with open(filename, 'rb') as f:
        bdata = []
        while True:
          bytes = f.read(2)
          if bytes == b'':
            break
          elif bytes == b'\xaa\xbb':
            endianess = "big"
            print("File endianess is 'Big Endian'")
            break
          elif bytes == b'\xbb\xaa':
            endianess = "little"
            print("File endianess is 'Little Endian'")
            break
          else:
            offset += 2
    return endianess, offset

def check_kind(filename, offset):
    kind = 0
    with open(filename, 'rb') as f:
        f.seek(offset,0)
        bytes = f.read(1)
        if bytes == b'\xa0':
            kind = "raw"
        else:
            kind = "compressed"
    return kind
        
def cread(filename):
    with open(filename, 'rb') as f:
         print("Reading calibration file {0}".format(filename))
         cal = pd.read_csv(filename, skiprows=18, header=None)
    return cal

def convert(path, cal_file, data_file, signal):
    evt = 0
    good = True
    data = np.array([-999])
    out = open(path + "/txt/" + data_file + ".csv", 'w')
    
    #Load calibration file
    cal = cread(path + cal_file)

    #Check for binary file endianess
    endianess, endianess_offset = seek_endianess(path + data_file)
  
    #Seek for event header offset
    header_offset = seek_header(path + data_file)

    #Read kind
    kind = check_kind(path + data_file, header_offset + 3)

    if kind == "raw":
        print("File {0} is raw data file".format(path + data_file))
    else:
        print("Error: file is not in raw data format")
        return
    
    #Read version
    _, version = bread(path + data_file, header_offset + endianess_offset -2, 1, endianess)
    if version[0] == "0x1212":
        print("File from miniTRB 6VA version")
        miniTRB_channels = 384
    elif version[0] == "0x1313":
        print("File from miniTRB 10VA version")
        miniTRB_channels = 640
    else:
        print("Error: invalid miniTRB version")
        return
    
    while good:
        bdata, hdata = bread(path + data_file, evt*1024 + header_offset + 4, miniTRB_channels, endianess)
        raw_values = np.array(bdata)
        if raw_values.shape[0] == miniTRB_channels:
            if signal:
                raw_values = np.concatenate((raw_values[::2],raw_values[1::2])) - cal[3].to_numpy()
                raw_values[cal[6].to_numpy() > 0] = 0
                np.savetxt(out,raw_values.reshape(1, miniTRB_channels).astype(float), fmt='%1.3f', delimiter=',')
                evt += 1
            else:
                raw_values = np.concatenate((raw_values[::2],raw_values[1::2]))
                raw_values[cal[6].to_numpy() > 0] = 0
                np.savetxt(out,raw_values.reshape(1, miniTRB_channels).astype(int), fmt='%i', delimiter=',')
                evt += 1
            print("Saving event {0}".format(evt), end='\r')
        else:
            good = False
            break
    out.close()       
    print("Found {0} events".format(evt))
    
    
    
def histo(path, cal_file, data_file):
    evt = 0
    good = True
    data = np.array([-999])
    
    #Load calibration file
    cal = cread(path + cal_file)

    #Check for binary file endianess
    endianess, endianess_offset = seek_endianess(path + data_file)
  
    #Seek for event header offset
    header_offset = seek_header(path + data_file)

    #Read kind
    kind = check_kind(path + data_file, header_offset + 3)

    if kind == "raw":
        print("File {0} is raw data file".format(path + data_file))
    else:
        print("Error: file is not in raw data format")
        return
    
    #Read version
    _, version = bread(path + data_file, header_offset + endianess_offset -2, 1, endianess)
    if version[0] == "0x1212":
        print("File from miniTRB 6VA version")
        miniTRB_channels = 384
    elif version[0] == "0x1313":
        print("File from miniTRB 10VA version")
        miniTRB_channels = 640
    else:
        print("Error: invalid miniTRB version")
        return
    
    while good:
        bdata, hdata = bread(path + data_file, evt*1024 + header_offset + 4, miniTRB_channels, endianess)
        raw_values = np.array(bdata)
        if raw_values.shape[0] == miniTRB_channels:
            raw_values = np.concatenate((raw_values[::2],raw_values[1::2])) - cal[3].to_numpy()
            data = np.append(data,raw_values.max())
            evt += 1
            print("Saving event {0}".format(evt), end='\r')
        else:
            good = False
            break
    print("Found {0} events".format(evt))
    fig=plt.figure(figsize=(16, 9), dpi= 80, facecolor='w', edgecolor='k')
    plt.hist(data, bins=250, range=(-10,500), log=False)

In [4]:
if __name__ == "__main__":
    convert("/home/gsilvest/Work/Dec2019_FOOT_data/HF000T/","draft/HF000T_PL_009.cal","data/HF000T_PL_007.dat", True)

Reading calibration file /home/gsilvest/Work/Dec2019_FOOT_data/HF000T/draft/HF000T_PL_009.cal
Reading data file /home/gsilvest/Work/Dec2019_FOOT_data/HF000T/data/HF000T_PL_007.dat
File endianess is 'Little Endian'
File /home/gsilvest/Work/Dec2019_FOOT_data/HF000T/data/HF000T_PL_007.dat is raw data file
File from miniTRB 6VA version
Found 1739 events


In [15]:
if __name__ == "__main__":
    histo("/home/gsilvest/Work/Dec2019_FOOT_data/HF000T/","draft/HF000T_PL_009.cal","data/HF000T_PL_011.dat")